In [ ]:
%%bash

pushd ../data

unzip -O CP936 -P D4nd8kHIfK xuelang_round1_test_b_20180802.zip.zip 
unzip -O CP936 -P Y5TzEQLbHD xuelang_round1_test_b.zip 

rm *.zip

popd

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import torch

from dataset import *
from model import *

In [ ]:
model = BaselineModel()
model.load_state_dict(torch.load("w.h5"))

In [ ]:
root_test = os.path.join("..", "data", "xuelang_round1_test_b")

import torchvision.transforms as transforms
transform_test = transforms.Compose([ 
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225)),
])

loader_test = get_loader(root_test, False, batch_size=32, transform=transform_test)

In [ ]:
import torch.nn.functional as F
import pandas as pd


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

def predict():
    result = pd.DataFrame(columns=["filename", "probability"])
    for x, path in loader_test:
        x = x.to(device)
    
        y = model(x)
        y = F.softmax(y, dim=1)
        # here should be 1 - y, but I realized it few days days later after my final submit
        # y = 1 - y.cpu().numpy()[:, 20]
        y = y.cpu().numpy()[:, 20]
        # in my final submit(which gives highest score), I convert prob to 0 and 1
        # but I quickly switch back and resubmit one without this conversion, I don't know which one it was used
        # y = (y > 0.5).astype("float32")
        data = pd.DataFrame({"filename":path, "probability":y})
        result = result.append(data, ignore_index=True)
    return result

with torch.no_grad():
    r = predict()

In [ ]:
r.head()

In [ ]:
r["probability"] = r["probability"].round(4).clip(0.0001, 0.9999)

In [ ]:
r.head()

In [ ]:
import datetime

filename = "../submit/submit_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + ".csv"
r.to_csv(filename, index=False, float_format='%.4f')